## BINARY CLASSIFICATION NOTEBOOK

This is a notebook intended for a semi automated binary classification problem. User inputs are explicitly called out, primarily around specific null treatment, feature encoding, sampling technique, modelling evaluation function. All of them have defaults, so only the mandatory inputs are listed here

***

### Contents
1. Pre-processing
2. Feature Engineering
3. Feature Selection
4. Modelling
    - xgboost
    - lightgbm
    - randomforest
    - simple logit
    - SVC
    - 1class methods
    - h2o AUTOML framework
    
***
- *Author* : VARUN V
- *Language* : python
- *Date* : 17-08-2017
***

In [8]:
## importing the relevant packages:

# clear the workspace
%reset -f

# print list of files in directory
import os
print(os.listdir())

# print/display all plots inline
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# the base packages
import collections # for the Counter function
import csv # for reading/writing csv files
import pandas as pd, numpy as np, time, gc, bisect

# the various packages/modules used across processing (sklearn), modelling (lightgbm) and bayesian optimization (hyperopt, bayes_opt)
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics, preprocessing, decomposition
from sklearn.cross_validation import cross_val_score, StratifiedKFold, StratifiedShuffleSplit
from sklearn.base import TransformerMixin
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
import category_encoders as ce

from bayes_opt import BayesianOptimization
from tqdm import tqdm
from hyperopt import hp, tpe, STATUS_OK, fmin, Trials
from hyperopt.fmin import fmin
from hyperopt.pyll.stochastic import sample

# modelling algorithms
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier

# Evaluation of the model
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from timeit import default_timer as timer

# Exporting packages for SHAP/LIME
import shap
import lime
import lime.lime_tabular

# missing value imputation
from fancyimpute import KNN, MICE #, NuclearNormMinimization

# define the global variables used later
MAX_EVALS = 2 # number of iterations/parameter sets created towards tuning (all hyper-opt frameworks)
N_FOLDS = 5 # number of cv folds
randomseed = 1 # the value for the random state used at various points in the pipeline

['.ipynb_checkpoints', 'FeatureEncoder.py', 'FEATURE_ENGINEERING.ipynb', 'MODEL_SELECTION_TUNING_TEST_2017Dec_model3.ipynb', 'OTHER_MODELS.ipynb', 'ULTIMATE.ipynb']


In [ ]:
x=pd.read_csv('')

In [192]:
x1=pd.read_csv('train_final.csv', na_values=['No Data', ' ', 'UNKNOWN'])
x2=pd.read_csv('test_final.csv', na_values=['No Data', ' ', 'UNKNOWN'])
x3=pd.read_csv('valid_final.csv', na_values=['No Data', ' ', 'UNKNOWN'])

print(x1.shape)
print(x2.shape)
print(x3.shape)

chars_to_remove = [' ', '.', '(', ')', '__', '-']
for i in chars_to_remove:
    x1.columns = x1.columns.str.strip().str.lower().str.replace(i, '_')
    x2.columns = x2.columns.str.strip().str.lower().str.replace(i, '_')
    x3.columns = x3.columns.str.strip().str.lower().str.replace(i, '_')

(6102, 92)
(5897, 92)
(5734, 92)


In [221]:
def num_missing(self):
    return sum(self.isnull())
x11=x1.apply(num_missing, axis=0)
x21=x2.apply(num_missing, axis=0)
x31=x3.apply(num_missing, axis=0)
missing = pd.DataFrame({'dec16': x11, 'june17': x21, 'dec17': x31}) # display count of NAs per column
missing['dec16_perc'] = 100*(missing['dec16']/x1.shape[0])
missing['june17_perc'] = 100*(missing['june17']/x2.shape[0])
missing['dec17_perc'] = 100*(missing['dec17']/x3.shape[0])

missing.index.name = 'columns'
missing.reset_index(inplace=True)

missing.to_csv('NAZ_missing_distribution.csv', index=False)

In [7]:
#### MAIN CLASSES ####

## Two defined for now ##
# 1. DataFrame Imputer
#    - for imputing missing values
# 2. Prepare Data
#    - for sourcing, processing, and returning the train/test datasets

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.
        Columns of dtype object are imputed with the most frequent value 
        in column.
        Columns of other types are imputed with mean of column.
        """
        
    def fit(self, X, y=None):
        """
        specify columns to be grouped by before imputation. use only for specific cases when user knows a specific level
        at which a column can be grouped at for missing value treatment
        
        X.groupby(['...'])
        """
        self.fill = pd.Series([X[c].value_counts().index[0] if X[c].dtype == np.dtype('O') else X[c].mean() for c in X], 
                              index=X.columns)
        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)
    
    def num_missing(self):
        return sum(self.isnull())
    
    def imputer_method(self, column, method=['mean', 'median', 'most_frequent']):
        x = Imputer(missing_values = 'NaN', strategy = method, axis = 0)
        return x.fit_transform(self[[column]]).ravel()
    
    def fancy_impute(X, which_method):
        """ currently supported algorithms are KNN, NNM and MICE from the fancyimpute package
        which_method = ['KNN', 'NNM', 'MICE']
        """
        if which_method == 'NNM': X = NuclearNormMinimization().complete(X) # NNM method
        if which_method == 'KNN': X = KNN(k=7, verbose=False).complete(X) # KNN method
        if which_method == 'MICE':
            X_complete_df = X.copy()
            mice = MICE(verbose=False)
            X_complete = mice.complete(np.asarray(X.values, dtype=float))
            X_complete_df.loc[:, X.columns] = X_complete[:][:]
            X = X_complete_df
        return X

class prepare_data():
    
    def __init__(self):
        """ To prepare data,
                1. read in data
                2. pre-processing/cleaning
                3. creating helper objects for later steps
                4. processing for modelling
                5. function return objects are the train, valid, response, categ cols/indices, feature names
        """
    
    def labelEncoder(train_df, valid_df, cat_columns, categorical_names):
        for feature in tqdm(cat_columns):
            le = preprocessing.LabelEncoder()
            le.fit(train_df[feature].astype(str))
            train_df[feature] = le.transform(train_df[feature].astype(str))
            valid_df[feature] = valid_df[feature].map(lambda i: 'No Data' if i not in le.classes_ else i)
            le_classes = le.classes_.tolist()
            bisect.insort_left(le_classes, 'No Data')
            le.classes_ = le_classes
            valid_df[feature] = le.transform(valid_df[feature].astype(str))
            categorical_names[feature] = le.classes_
            return train_df, valid_df, categorical_names
        
    def ce_encodings(train_df, valid_df, y_train, y_valid, cat_columns, encoding):
        if encoding=='bne':
            enc1=ce.BaseNEncoder(cat_columns, base=3)
            enc1.fit(train_df)
            train_df=enc1.transform(train_df)
            valid_df=enc1.transform(valid_df)
        elif encoding=='be':
            enc2=ce.BinaryEncoder(cat_columns)
            enc2.fit(train_df)
            train_df=enc2.transform(train_df)
            valid_df=enc2.transform(valid_df)
        elif encoding=='he':
            enc3=ce.HashingEncoder(cat_columns)
            enc3.fit(train_df, y_train)
            train_df=enc3.transform(train_df)
            valid_df=enc3.transform(valid_df)
        elif encoding=='oe':
            enc4=ce.OrdinalEncoder(cat_columns)
            enc4.fit(train_df, y_train)
            train_df=enc4.transform(train_df)
            valid_df=enc4.transform(valid_df)
        return train_df, valid_df
    
    ## function to get frequency count of elements in a vector/list
    def freq_count(input_vector):
        return collections.Counter(input_vector)
    
    def categ_feats(train_df, valid_df, y_train, y_valid, encoding='le'):
        x = list(train_df.dtypes)
        x_1 = [1 if x == 'O' else 0 for x in x]
        categorical_idx = [i for i, x in enumerate(x_1) if x == 1]
        # Get feature names and their values for categorical data (needed for LIME)
        cat_columns = train_df.select_dtypes(include=['object']).columns.values
        categorical_names = {}

        if encoding=='le':
            train_df, valid_df, categorical_names = prepare_data.labelEncoder(train_df, valid_df, cat_columns, categorical_names)
        elif encoding in ['be', 'bne', 'he', 'oe']:
            train_df, valid_df = prepare_data.ce_encodings(train_df, valid_df, y_train, y_valid, cat_columns, encoding)
        else :
            print('Not supported. Use one of [be, bne, he, oe]')
        return train_df, valid_df, categorical_names, categorical_idx

    def create(input_file_path, input_file_path_2, response, cols_to_remove = ['id'], random_seed = 1,
                            encoding = 'le'):
        train = pd.read_csv(input_file_path, na_values=['No Data', ' ', 'UNKNOWN'])
        test = pd.read_csv(input_file_path_2, na_values=['No Data', ' ', 'UNKNOWN'])
        
        train.drop(cols_to_remove, axis = 1, inplace = True)
        test = pd.DataFrame(data = test[train.columns])
        
        chars_to_remove = [' ', '.', '(', ')', '__', '-']
        for i in chars_to_remove:
            train.columns = train.columns.str.strip().str.lower().str.replace(i, '_')
            test.columns = test.columns.str.strip().str.lower().str.replace(i, '_')
            
        print(train.shape, '\n')
        train.dropna(thresh=0.5*(train.shape[0]), axis=1, inplace = True)
        train.dropna(thresh=0.4*(train.shape[1]), axis=0, inplace = True)
        print(train.shape, '\n')
        test = test[train.columns]
        test.dropna(thresh=0.5*(test.shape[0]), axis=1, inplace = True)
        train = train[test.columns]
        
        print(prepare_data.freq_count(train[response]), '\n')

        # shuffle the dataframes so that the training is done in a random order.
        train = shuffle(train)
        test = shuffle(test)
        
        # creating the response vector
        y_train = train[response].values
        X_train = train.drop([response], axis = 1)
        y_valid = test[response].values
        X_valid = test.drop([response], axis = 1)
        
        X_train = pd.DataFrame(X_train)
        X_valid = pd.DataFrame(X_valid)
        
        X_train, X_valid, categ_names, categ_idx = prepare_data.categ_feats(X_train, X_valid, y_train, y_valid, encoding)
        
        feat_names = X_train.columns.values
        feat_names2 = X_valid.columns.values
        
        X_train = DataFrameImputer.fancy_impute(X_train, which_method='MICE')
        X_valid = DataFrameImputer.fancy_impute(X_valid, which_method='MICE')
        
        # returning as pandas dataframes to retain feature names for LIME and feature importance plots
        X_train = pd.DataFrame(data=X_train, columns=feat_names)
        X_valid = pd.DataFrame(data=X_valid, columns=feat_names2)
        
        return X_train, X_valid, y_train, y_valid, categ_names, categ_idx, feat_names

In [189]:
# create data function call
# CV approach
# train and valid features/response dataframes returned
# categorical column names/indices and all feature names also returned

X_train, X_valid, y_train, y_valid, categ_names, categ_idx, feat_names = prepare_data.create(input_file_path='../input/adult.data',
                                                                  input_file_path_2='../input/adult.test.csv', response = 'label',
                                cols_to_remove = [], encoding = 'bne')

(6102, 77) 

(6085, 75) 

Counter({0: 5796, 1: 289}) 

['pay_scale_group_0' 'pay_scale_group_1' 'pay_scale_group_2'
 'pay_scale_group_3' 'gender_0' 'gender_1' 'abinbev_entity2_0'
 'abinbev_entity2_1' 'abinbev_entity2_2' 'abinbev_entity2_3'
 'macro_entity4_0' 'macro_entity4_1' 'macro_entity4_2' 'macro_entity4_3'
 'macro_entity4_4' 'physical_work_location_code_0'
 'physical_work_location_code_1' 'physical_work_location_code_2'
 'physical_work_location_code_3' 'physical_work_location_code_4'
 'physical_work_location_code_5' 'global_job_0' 'global_job_1'
 'global_job_2' 'global_job_3' 'global_job_4' 'global_job_5'
 'global_job_6' 'job_family_0' 'job_family_1' 'job_family_2'
 'job_family_3' 'job_family_4' 'job_family_5' 'job_family_6'
 'functional_area_0' 'functional_area_1' 'functional_area_2'
 'functional_area_3' 'functional_area_4' 'functional_area_5'
 'cost_center_0' 'cost_center_1' 'cost_center_2' 'cost_center_3'
 'cost_center_4' 'cost_center_5' 'cost_center_6' 'macro_entity_type_0'
 '

ValueError: Input matrix is not missing any values

In [155]:
print(X_train.shape)
#print(X_test.shape)
print(X_valid.shape)

print(y_train.shape)
#print(y_test.shape)
print(y_valid.shape)

print(collections.Counter(y_train))
#print(collections.Counter(y_test))
print(collections.Counter(y_valid))

(6085, 74)
(5734, 74)
(6085,)
(5734,)
Counter({0: 5796, 1: 289})
Counter({0: 5453, 1: 281})


# FEATURE ENGINEERING MODULE

1. PCA
2. ICA
3. tSVD
4. GRP
5. SRP
6. Binning
7. Deviation Encoding features
8. Salary related features
9. ...

In [158]:
class feat_eng():
    
    def __init__():
        """ this module contains several functions for creating new features. find below a brief description of each """
    
    def scalers(train, valid, which_method):
        if which_method == 'ss':
            sc = StandardScaler()
            sc.fit(train)
            train = pd.DataFrame(sc.transform(train))
            valid = pd.DataFrame(sc.transform(valid))
            return train, valid # scale all variables to zero mean and unit variance, required for PCA and related
        if which_method == 'mm':
            mm = MinMaxScaler()
            mm.fit(train)
            train = pd.DataFrame(mm.transform(train))
            valid = pd.DataFrame(mm.transform(valid))
            return train, valid # use this method to iterate
        
    def pca_feats(train, valid, n = .95):
            train, valid = feat_eng.scalers(train, valid, which_method='ss')
            pca_fit = decomposition.PCA(n_components=n)
            pca_fit.fit(train)
            pca_train = pd.DataFrame(pca_fit.transform(train))
            pca_valid = pd.DataFrame(pca_fit.transform(valid))
            pca_cols = list(set(list(pca_train)))
            pca_cols = ['pca_' + str(s) for s in pca_cols]
            pca_train.columns = pca_cols
            pca_valid.columns = pca_cols
            return pca_train, pca_valid
        
    def ica_feats(train, valid, n = 5):
            train, valid = feat_eng.scalers(train, valid, which_method='ss')
            ica_fit = decomposition.FastICA(n_components=n)
            ica_fit.fit(train)
            ica_train = pd.DataFrame(ica_fit.transform(train))
            ica_valid = pd.DataFrame(ica_fit.transform(valid))
            ica_cols = list(set(list(ica_train)))
            ica_cols = ['ica_' + str(s) for s in ica_cols]
            ica_train.columns = ica_cols
            ica_valid.columns = ica_cols
            return ica_train, ica_valid
        
    def tsvd_feats(train, valid, n = 5):
            train, valid = feat_eng.scalers(train, valid, which_method='ss')
            tsvd_fit = decomposition.TruncatedSVD(n_components=n)
            tsvd_fit.fit(train)
            tsvd_train = pd.DataFrame(tsvd_fit.transform(train))
            tsvd_valid = pd.DataFrame(tsvd_fit.transform(valid))
            tsvd_cols = list(set(list(tsvd_train)))
            tsvd_cols = ['tsvd_' + str(s) for s in tsvd_cols]
            tsvd_train.columns = tsvd_cols
            tsvd_valid.columns = tsvd_cols
            return tsvd_train, tsvd_valid
        
    def grp_feats(train, valid, n = 5):
            train, valid = feat_eng.scalers(train, valid, which_method='ss')
            grp_fit = GaussianRandomProjection(n_components=n, eps=0.1)
            grp_fit.fit(train)
            grp_train = pd.DataFrame(grp_fit.transform(train))
            grp_valid = pd.DataFrame(grp_fit.transform(valid))
            grp_cols = list(set(list(grp_train)))
            grp_cols = ['grp_' + str(s) for s in grp_cols]
            grp_train.columns = grp_cols
            grp_valid.columns = grp_cols
            return grp_train, grp_valid
    
    def srp_feats(train, valid, n = 5):
            train, valid = feat_eng.scalers(train, valid, which_method='ss')
            srp_fit = SparseRandomProjection(n_components=n, dense_output=True, eps=0.1)
            srp_fit.fit(train)
            srp_train = pd.DataFrame(srp_fit.transform(train))
            srp_valid = pd.DataFrame(srp_fit.transform(valid))
            srp_cols = list(set(list(srp_train)))
            srp_cols = ['srp_' + str(s) for s in srp_cols]
            srp_train.columns = srp_cols
            srp_valid.columns = srp_cols
            return srp_train, srp_valid
        
    def return_combined(train, valid, list_objects = ['pca', 'ica', 'tsvd', 'grp', 'srp']):
        if 'pca' in list_objects:
            train = pd.concat([train.reset_index(drop=True), pca_train], axis=1)
            valid = pd.concat([valid.reset_index(drop=True), pca_valid], axis=1)
        if 'ica' in list_objects:
            train = pd.concat([train.reset_index(drop=True), ica_train], axis=1)
            valid = pd.concat([valid.reset_index(drop=True), ica_valid], axis=1)
        if 'tsvd' in list_objects:
            train = pd.concat([train.reset_index(drop=True), tsvd_train], axis=1)
            valid = pd.concat([valid.reset_index(drop=True), tsvd_valid], axis=1)
        if 'grp' in list_objects:
            train = pd.concat([train.reset_index(drop=True), grp_train], axis=1)
            valid = pd.concat([valid.reset_index(drop=True), grp_valid], axis=1)
        if 'srp' in list_objects:
            train = pd.concat([train.reset_index(drop=True), srp_train], axis=1)
            valid = pd.concat([valid.reset_index(drop=True), srp_valid], axis=1)
        return train, valid

In [159]:
## calling the various feat engineering functions and adding those features
## pca, ica, tsvd, grp, srp
pca_train, pca_valid = feat_eng.pca_feats(train=X_train, valid=X_valid, n=.95)
ica_train, ica_valid = feat_eng.ica_feats(train=X_train, valid=X_valid, n=10)
tsvd_train, tsvd_valid = feat_eng.tsvd_feats(train=X_train, valid=X_valid, n=10)
grp_train, grp_valid = feat_eng.grp_feats(train=X_train, valid=X_valid, n=10)
srp_train, srp_valid = feat_eng.srp_feats(train=X_train, valid=X_valid, n=10)

## scale the data
X_train, X_valid = feat_eng.scalers(train=X_train, valid=X_valid, which_method='mm')

## return the final datasets with the added features
X_train, X_valid = feat_eng.return_combined(train = X_train, valid = X_valid)

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\decomposition\fastica_.py:118: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


In [160]:
X_train['response'] = y_train
X_valid['response'] = y_valid

X_train.to_csv('X_train.csv', index=False)
X_valid.to_csv('X_valid.csv', index=False)

In [168]:
## h2o AUTO_ML grid search framework
import h2o
from h2o.automl import H2OAutoML

class h2o_automl():
    
    def __init__():
        """ module to invoke h2o auto ml and tune on the sampling parameters. 
        will need to incorporate to super pipeline later """
        
    def score(params):        
        # function to be minimized (1 - ROCAUC)
        global ITERATION
        ITERATION += 1
        randomseed = 1
        
        #h2o_train = h2o.H2OFrame(params['h2o_train'])
        #h2o_valid = h2o.H2OFrame(params['h2o_valid'])
        
        aml = H2OAutoML(max_runtime_secs = params['time_to_run'], stopping_metric='mean_per_class_error', sort_metric='mean_per_class_error',
                        class_sampling_factors=[params['oversampling'], params['undersampling']],
                        balance_classes = params['balance_classes'])

        aml.train(y = 'response', training_frame = h2o_train)
        
        # Print Leaderboard (ranked by xval metrics)
        print(aml.leaderboard)
        # Evaluate performance on a test set
        perf = aml.leader.model_performance(h2o_valid)
        print(perf.auc())   
        
        return {'loss': (1 - perf.auc()), 'status': STATUS_OK, 'params': params, 'iteration': ITERATION, 'aml': aml}
    
    def auto_ml(train_path, valid_path, response = 'response', time_to_run = 30):
        # Keep track of evals
        global ITERATION
        ITERATION = 0
        global trials
        trials = Trials()
        global h2o_train, h2o_valid
        
        # initializing the h2o cluster
        h2o.init()
        
        # Import a sample binary outcome train/test set into H2O
        h2o_train = h2o.import_file(train_path, header=1)
        h2o_valid = h2o.import_file(valid_path, header=1)

        # Identify the response and set of predictors
        x = list(h2o_train.columns)  #if x is defined as all columns except the response, then x is not required
        x.remove(response)

        # For binary classification, response should be a factor
        h2o_train[response] = h2o_train[response].asfactor()
        h2o_valid[response] = h2o_valid[response].asfactor()

        # space to be traversed for the hyperopt function
        space = {
            'undersampling': hp.uniform('us', 0.01, 1),
            'oversampling': hp.uniform('os', 1, 10),
            'balance_classes': hp.choice('bc', ['True', 'False']),
            'time_to_run': 30, 'x': x, 'response': response}

        best = fmin(h2o_automl.score, space, algo=tpe.suggest, trials=trials, max_evals=MAX_EVALS,
                    rstate=np.random.RandomState(randomseed))
        best = trials.best_trial['result']['params']
        aml = trials.best_trial['result']['aml']
        
        return trials, best, aml # results of all the iterations
    
    def get_score(aml, h2o_valid, y_valid, threshold = 0.1):
        pred = aml.predict(h2o_valid)[:,2]
        pred = pred.as_data_frame().as_matrix()
        predict = np.where(pred > threshold, 1, 0)
        y_test=y_valid

        recall_score = sklearn.metrics.recall_score(y_pred=predict, y_true=y_test)
        precision_score = sklearn.metrics.precision_score(y_pred=predict, y_true=y_test)
        f1_score = sklearn.metrics.f1_score(y_pred=predict, y_true=y_test)
        auc_score = roc_auc_score(y_test, pred)
        tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_pred=predict, y_true=y_test).ravel()
        print(sklearn.metrics.confusion_matrix(y_pred=predict, y_true=y_test), '\n')
        print('recall score is: ', recall_score)
        print('precision score is: ', precision_score)
        print('f1_score is: ', f1_score)
        print('accuracy score: ', sklearn.metrics.accuracy_score(y_true=y_test, y_pred=predict))
        print('The final AUC after taking the best params and num_rounds when it stopped is {:.4f}.'.format(auc_score), '\n')

In [224]:
trials, best, aml = h2o_automl.auto_ml(train_path='X_train.csv', valid_path='X_valid.csv', response='response', time_to_run=60)

print(aml.leader.model_performance(h2o_valid))

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_0_AutoML_20180814_180357_model_2


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.012345731874956202
RMSE: 0.11111134899260382
LogLoss: 0.05451098842472014
Mean Per-Class Error: 0.013020362902665505
AUC: 0.9970281519607433
Gini: 0.9940563039214867
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.1987366596826228: 


,0,1,Error,Rate
0,4668.0,5.0,0.0011,(5.0/4673.0)
1,12.0,219.0,0.0519,(12.0/231.0)
Total,4680.0,224.0,0.0035,(17.0/4904.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.1987367,0.9626374,172.0
max f2,0.1525126,0.9664948,182.0
max f0point5,0.2089268,0.9740840,170.0
max accuracy,0.2089268,0.9965334,170.0
max precision,0.9251269,1.0,0.0
max recall,0.0167882,1.0,343.0
max specificity,0.9251269,1.0,0.0
max absolute_mcc,0.1987367,0.9609453,172.0
max min_per_class_accuracy,0.0959240,0.9869463,207.0
max mean_per_class_accuracy,0.0959240,0.9869796,207.0


Gains/Lift Table: Avg response rate:  4.71 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2018-08-14 18:04:55,12.264 sec,0.0,0.2118622,0.1898994,0.5,1.0,0.9528956,0.2161089,0.1959333,0.5,1.0,0.9508891
,2018-08-14 18:04:56,12.528 sec,5.0,0.1912917,0.1436173,0.9263643,18.2573160,0.0391517,0.2131652,0.1861076,0.6754076,5.0905172,0.0982218
,2018-08-14 18:04:56,12.836 sec,10.0,0.1692332,0.1106636,0.9760876,20.8048485,0.0203915,0.2114478,0.1813669,0.7087159,8.4841954,0.0685859
,2018-08-14 18:04:56,13.132 sec,15.0,0.1546803,0.0932565,0.9871292,21.2294372,0.0134584,0.2111540,0.1813863,0.6947524,8.4841954,0.1278577
,2018-08-14 18:04:56,13.416 sec,20.0,0.1424780,0.0806138,0.9919418,21.2294372,0.0103997,0.2118183,0.1826487,0.7049698,10.1810345,0.0499577
,2018-08-14 18:04:57,13.676 sec,25.0,0.1312188,0.0704007,0.9942231,21.2294372,0.0065253,0.2118712,0.1838461,0.6983450,6.7873563,0.0770533
,2018-08-14 18:04:57,13.936 sec,30.0,0.1201879,0.0611406,0.9963440,21.2294372,0.0048940,0.2121802,0.1844297,0.7119707,8.4841954,0.0736664
,2018-08-14 18:04:57,14.188 sec,34.0,0.1111113,0.0545110,0.9970282,21.2294372,0.0034666,0.2124073,0.1856861,0.7111800,8.4841954,0.0804403


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
2,15.0153389,1.0,0.0307435
71,12.2121620,0.8133124,0.0250040
pca_12,11.1265478,0.7410121,0.0227813
pca_0,10.4576960,0.6964675,0.0214118
3,10.4030600,0.6928288,0.0213000
---,---,---,---
59,0.0,0.0,0.0
60,0.0,0.0,0.0
62,0.0,0.0,0.0
64,0.0,0.0,0.0



See the whole table with table.as_data_frame()



In [ ]:
h2o_automl.get_score(aml=aml, h2o_valid=h2o_valid, y_valid=y_valid, threshold=0.1)

In [228]:
h2o.cluster().shutdown()

H2O session _sid_bd02 closed.
